In [35]:
import pystan
import numpy as np
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
from collections import Counter
from scipy.special import digamma
from scipy.stats import gamma
from scipy.special import gammaln
from scipy.special import multigammaln
from sklearn.mixture import GaussianMixture
import time

from matplotlib import pyplot as plt

## for simulation
from numpy.random import dirichlet
from numpy.random import multinomial
from numpy.random import multivariate_normal
#from scipy.stats import wishart

In [24]:
gene_code = """
data{
    
// observations

    int<lower=2> K;     // Number of modules
    int<lower=2> V;     // Number of features (dimensions)
    int<lower=2> M;     // Number of genes
    int<lower=2> N;     // Number of regions for all genes
    
    vector[V] w[N];     // Observations (region x features)
    int<lower=1,upper=M> gene[N];    // gene ID for region n
    

// priors

    real alpha;  
    
    real nu;
    cov_matrix[V] W;
    vector[V] m;
    real beta;
}
    

    
transformed data{
    vector<lower=0>[K] alpha_vec;
    for (k in 1:K){
        alpha_vec[k] = alpha; //symmetric dirichlet prior
    }
}


parameters{
    simplex[K] theta[M];
    vector[V] mu[K];
    cov_matrix[V] lambda[K];
}

transformed parameters{
    cov_matrix[V] inv_lambda[K];
    for (k in 1:K){
        inv_lambda[k] = inv(lambda[k]);
    }
}



    
model{

// priors
    for (d in 1:M)
        theta[d] ~ dirichlet(alpha_vec);
    for (k in 1:K){
        lambda[k] ~ wishart(nu, W);
        mu[k] ~ multi_normal_prec(m, beta * lambda[k]);
    }

// log marginal (summing over all z) probability of the inputs
    for (n in 1:N){
        real gamma[K];
        for (k in 1:K)
            gamma[k] = log(theta[gene[n],k]) + multi_normal_lpdf(w[n]|mu[k], inv_lambda[k]);  
        target += log_sum_exp(gamma);   // multiplies the probabilities for each state k on the log scale
    }
}
"""

In [34]:
def generate_data(fn):
    ## topics for each document
    alphai = 0.25  ## prior of Dirichlet process
    M = 20  ## number of genes
    K = 4   ## number of clusters
    Nd = np.random.choice(range(150,250),M)   ## number of regions in each gene
    N = np.sum(Nd)   ## number of total regions
    V = 124
    theta = dirichlet(np.ones(K)*alphai, M)
    ## cluster assignment for each region
    word_assginment_z = []
    for d in xrange(M):
        word_assginment_z.append([multinomial(1, theta[d]) for t in xrange(Nd[d])])
        word_assginment_z[d] = np.array(word_assginment_z[d])
    ## parameter for the modules
    nu0 = V + 5.0
    m0 = np.zeros(V)
    beta0 = 1.0
    Lambbda = np.load('VI/%s' % fn)['arr_0']
    mu = [multivariate_normal(m0, np.linalg.inv(beta0 * Lambbda[k]), size=1) for k in xrange(K)]
    ## generate word values
    words_values = []
    for d in xrange(M):
        words_values.append([])
        for n in xrange(Nd[d]):
            zzz = np.where(word_assginment_z[d][n,:])[0][0]
            words_values[d].append(multivariate_normal(mu[zzz][0], np.linalg.inv(Lambbda[zzz]), size = 1)[0])
        words_values[d] = np.array(words_values[d])
    print 'Determinant of the data covariance:', np.linalg.det(np.cov(np.transpose(np.array([a for b in words_values for a in b]))))
    print 'Conditional number of the data covairance:', np.linalg.cond(np.cov(np.transpose(np.array([a for b in words_values for a in b]))))
    dat = {'alpha': alphai,
                   'nu': nu0,
                   'W': np.identity(V),
                   'm': m0,
                   'beta': beta0,    
                   'K': K,
                   'V': V,
                   'M': M,
                   'N': np.sum(Nd),               
                   'w': [a for b in words_values for a in b],
                   'gene': [t for i, j in zip(range(1,M+1), Nd) for t in np.repeat(i,j)]}
    return dat


In [33]:
gene_dat = generate_data('Simulated_Lambbda_small_W0_large_data_std.npz')
fit = pystan.stan(model_code=gene_code, data=gene_dat, iter=2, chains=1)

4.20060169246e-153
1408.50257017
